In [1]:
import sys
import random as r
import openpyxl
from openpyxl import Workbook
import matplotlib.pyplot as plt
from PyQt5.QtWidgets import (QApplication, QWidget, QVBoxLayout, QHBoxLayout, QPushButton, QLabel, 
                             QInputDialog, QMessageBox)
from PyQt5.QtCore import Qt
from PyQt5.QtWidgets import QComboBox, QDialog, QVBoxLayout, QDialogButtonBox, QLineEdit


# Función para registrar resultados en Excel
def registrar_resultado(nombre_jugador, resultado, intentos):
    try:
        workbook = openpyxl.load_workbook("Resultados.xlsx")
        sheet = workbook.active
    except FileNotFoundError:
        workbook = Workbook()
        sheet = workbook.active
        sheet.append(["Jugador", "Resultado", "Intentos"])

    sheet.append([nombre_jugador, resultado, intentos])
    workbook.save("Resultados.xlsx")
    workbook.close()

# Función para mostrar estadísticas
def mostrar_estadisticas():
    try:
        workbook = openpyxl.load_workbook("Resultados.xlsx")
        sheet = workbook.active
        jugadores = []
        ganados = []
        perdidos = []

        for row in sheet.iter_rows(min_row=2, values_only=True):
            jugador = row[0]
            resultado = row[1]

            if jugador not in jugadores:
                jugadores.append(jugador)
                ganados.append(0)
                perdidos.append(0)

            idx = jugadores.index(jugador)
            if resultado == "Ganó":
                ganados[idx] += 1
            elif resultado == "Perdió":
                perdidos[idx] += 1

        if jugadores:
            fig, ax = plt.subplots()
            bar_width = 0.35
            index = range(len(jugadores))

            ax.bar(index, ganados, bar_width, color='green', label='Ganó')
            ax.bar([i + bar_width for i in index], perdidos, bar_width, color='red', label='Perdió')

            ax.set_xlabel('Jugador')
            ax.set_ylabel('Número de Resultados')
            ax.set_title('Resultados de Partidas')
            ax.set_xticks([i + bar_width / 2 for i in index])
            ax.set_xticklabels(jugadores)
            ax.legend(title="Resultado")

            plt.xticks(rotation=45, ha="right")
            plt.tight_layout()
            plt.show()
        else:
            QMessageBox.information(None, "Estadísticas", "No hay datos suficientes para mostrar estadísticas.")
        
        workbook.close()

    except FileNotFoundError:
        QMessageBox.information(None, "Estadísticas", "No hay estadísticas registradas aún.")

# Función para seleccionar jugador
def seleccionar_jugador():
    try:
        # Abre el archivo y obtiene la lista de jugadores únicos
        workbook = openpyxl.load_workbook("Resultados.xlsx")
        sheet = workbook.active
        jugadores = list({sheet[f"A{row}"].value for row in range(2, sheet.max_row + 1) if sheet[f"A{row}"].value})
        workbook.close()
    except FileNotFoundError:
        jugadores = []

    # Diálogo personalizado para seleccionar o ingresar un nombre nuevo
    dialog = QDialog()
    dialog.setWindowTitle("Seleccionar Jugador")

    layout = QVBoxLayout()
    combo_box = QComboBox()
    combo_box.addItems(jugadores)
    combo_box.setEditable(True)  # Permite ingresar un nombre nuevo en el cuadro de texto
    layout.addWidget(QLabel("Selecciona un jugador o ingresa un nombre nuevo:"))
    layout.addWidget(combo_box)

    # Botones de aceptar o cancelar
    buttons = QDialogButtonBox(QDialogButtonBox.Ok | QDialogButtonBox.Cancel)
    buttons.accepted.connect(dialog.accept)
    buttons.rejected.connect(dialog.reject)
    layout.addWidget(buttons)

    dialog.setLayout(layout)

    if dialog.exec() == QDialog.Accepted:
        nombre_jugador = combo_box.currentText().strip()
        if nombre_jugador:
            return nombre_jugador

    return None  # Si se cancela o no se ingresa un nombre, retorna None

# Función para seleccionar dificultad
def Dificultad(numero):
    dificultad, ok = QInputDialog.getInt(None, "Dificultad", "Seleccione una dificultad:\n1. Fácil (20 intentos)\n2. Difícil (12 intentos)\n3. Muy Difícil (5 intentos)\n4. Sugerida", min=1, max=4)
    if not ok:
        return None

    if dificultad == 1:
        return 20
    elif dificultad == 2:
        return 12
    elif dificultad == 3:
        return 5
    elif dificultad == 4:
        if numero % 100 == 0 or numero == 1 or numero == 1000:
            return 5
        elif numero % 5 == 0 or numero in range(1, 100):
            return 12
        else:
            return 20

# Función para jugar adivinanza
def Adivina(d, numero):
    intentos_iniciales = d
    while d > 0:
        adivina, ok = QInputDialog.getInt(None, "Adivina el Número", f"Adivina el número entre 1 y 1000. Tienes {d} intentos:")
        if not ok:
            break

        if adivina == numero:
            QMessageBox.information(None, "¡Bien hecho!", "¡Adivinaste el número!")
            return intentos_iniciales - d + 1
        elif adivina > numero:
            QMessageBox.information(None, "Pista", "El número buscado es menor.")
        else:
            QMessageBox.information(None, "Pista", "El número buscado es mayor.")
        d -= 1

    QMessageBox.information(None, "Juego terminado", f"Se acabaron los intentos. El número era: {numero}")
    return -1

# Función para iniciar partida en modo solitario
def partida_solitario():
    numero = r.randint(1, 1000)
    dificultad = Dificultad(numero)
    if dificultad:
        intentos_usados = Adivina(dificultad, numero)
        nombre_jugador = seleccionar_jugador()
        if nombre_jugador:
            if intentos_usados != -1:
                registrar_resultado(nombre_jugador, "Ganó", intentos_usados)
            else:
                registrar_resultado(nombre_jugador, "Perdió", dificultad)

# Función para iniciar partida de 2 jugadores
def partida_dos_jugadores():
    numero, ok = QInputDialog.getInt(None, "Jugador 1", "Introduce un número entre 1 y 1000:", min=1, max=1000)
    if not ok:
        return

    dificultad = Dificultad(numero)
    if dificultad:
        intentos_usados = Adivina(dificultad, numero)
        nombre_jugador = seleccionar_jugador()
        if nombre_jugador:
            if intentos_usados != -1:
                registrar_resultado(nombre_jugador, "Ganó", intentos_usados)
            else:
                registrar_resultado(nombre_jugador, "Perdió", dificultad)

# Clase principal de la aplicación
class AdivinaNumeroApp(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):
        self.setWindowTitle("Juego: Adivina el Número")
        layout = QVBoxLayout()

        # Botones del menú
        btn_solitario = QPushButton("Partida modo solitario")
        btn_solitario.clicked.connect(partida_solitario)
        layout.addWidget(btn_solitario)

        btn_dos_jugadores = QPushButton("Partida 2 Jugadores")
        btn_dos_jugadores.clicked.connect(partida_dos_jugadores)
        layout.addWidget(btn_dos_jugadores)

        btn_estadisticas = QPushButton("Estadísticas")
        btn_estadisticas.clicked.connect(mostrar_estadisticas)
        layout.addWidget(btn_estadisticas)

        btn_salir = QPushButton("Salir")
        btn_salir.clicked.connect(self.close)
        layout.addWidget(btn_salir)

        self.setLayout(layout)

# Ejecutar la aplicación
if __name__ == '__main__':
    app = QApplication(sys.argv)
    ventana = AdivinaNumeroApp()
    ventana.show()
    app.exec_()

